In [2]:
!pip install -q qwen-vl-utils
!pip install -q git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830
!pip install -q accelerate
!pip -q install sentence-transformers==3.0.1

In [ ]:
!git clone https://github.com/qphat289/finetune_qwenvl2_with_llamafactory.git

In [2]:
%cd /kaggle/working/finetune_qwenvl2_with_llamafactory
!pwd

/kaggle/working/finetune_qwenvl2_with_llamafactory
/kaggle/working/finetune_qwenvl2_with_llamafactory


In [ ]:
!pip install bitsandbytes
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/transformers.git
!pip install -e ".[torch, metrics]"
!pip install liger-kernel

# DATA PREPARE

In [3]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn.functional as F
import argparse
import numpy as np
import json

from IPython.display import clear_output as cls
from IPython.display import FileLink

import time
from tqdm.auto import tqdm
import json
from io import BytesIO
import re
from torch import nn
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

In [5]:
from datasets import load_dataset
import pandas as pd
import os
import torch
from PIL import Image

ds = load_dataset("Vi-VLM/Vista", "vi_llava_complex_reasoning", split='train')
root_image = '/kaggle/input/coco-2017-dataset/coco2017/train2017'

data = []
for i in ds:
    try:
        data.append({
            "messages": [
                    {
                    "content": "<image>" + i['conversation'][0]['content'],
                    "role": "user"
                    },
                    {
                    "content": i['conversation'][1]['content'],
                    "role": "assistant"
                    }
                ],
            "images": [
                "data/vista/" + i['file_name']
            ]
        }
        )
            
#             'question_id': i['id'],
#             'image_id': os.path.join(root_image,i['file_name']),
#             'question': i['conversation'][0]['content'],
#             'answer': i['conversation'][1]['content']
#         }])
    except IndexError:
        print(f"Wrong format at id {i['id']}")
        continue

Wrong format at id 236182
Wrong format at id 320078
Wrong format at id 65209
Wrong format at id 29140
Wrong format at id 200857
Wrong format at id 481810
Wrong format at id 525152
Wrong format at id 332296
Wrong format at id 238390
Wrong format at id 572985
Wrong format at id 11373
Wrong format at id 520778
Wrong format at id 54899
Wrong format at id 119974
Wrong format at id 36082
Wrong format at id 547487
Wrong format at id 286149
Wrong format at id 363718
Wrong format at id 563178
Wrong format at id 66067
Wrong format at id 26809


/kaggle/input/coco-2017-dataset/coco2017/train2017/

In [6]:
data[2]

{'messages': [{'content': '<image>Ngoài các thiết bị được nhìn thấy rõ ràng, liệu căn bếp này có đủ chỗ cho những thiết bị nào khác không?',
   'role': 'user'},
  {'content': 'Bếp rất hẹp và không có đủ chỗ cho các thiết bị khác. Cả hai bên của nhà bếp đều có tủ và thiết bị, và không có không gian trống để đặt thêm thiết bị. Ngoài ra, tủ lạnh và lò nướng đã chiếm phần lớn không gian trên tường, không còn chỗ cho các thiết bị lớn hơn.',
   'role': 'assistant'}],
 'images': ['data/vista/000000403013.jpg']}

In [7]:
import json
json_file = 'newest_qwenvl2.json'
with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

# Train with LLaMA Factory

In [1]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
import json

args = dict(
  stage="sft",                        # do supervised fine-tuning
  do_train=True,
  model_name_or_path="Qwen/Qwen2-VL-2B-Instruct", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="vista_data",             # use alpaca and identity datasets
  template="qwen2_vl",                     # use llama3 prompt template
  finetuning_type="lora",                   # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir="qwen2vl_lora",                  # the path to save LoRA adapters
  per_device_train_batch_size=2,               # the batch size
  gradient_accumulation_steps=4,               # the gradient accumulation steps
  lr_scheduler_type="cosine",                 # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=20,                      # save checkpoint every 1000 steps
  learning_rate=5e-5,                     # the learning rate
  num_train_epochs=3.0,                    # the epochs of training
  max_samples=500,                      # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  loraplus_lr_ratio=16.0,                   # use LoRA+ algorithm with lambda=16.0
  fp16=True,                         # use float16 mixed precision training
  use_liger_kernel=True,                   # use liger kernel for efficient training
)

In [10]:
json.dump(args, open("train_qwen2vl.json", "w", encoding="utf-8"), indent=2)

In [4]:
!llamafactory-cli train train_qwen2vl.json

'llamafactory-cli' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
args = dict(
  model_name_or_path="Qwen/Qwen2-VL-2B-Instruct", # use official non-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="qwen2vl_lora",            # load the saved LoRA adapters
  template="qwen2_vl",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  export_dir="qwen2vl_2b_instruct_lora_merged",              # the path to save the merged model
  export_size=2,                       # the file shard size (in GB) of the merged model
  export_device="cuda",                    # the device used in export, can be chosen from `cpu` and `cuda`
  #export_hub_model_id="your_id/your_model",         # the Hugging Face hub ID to upload model
)


In [14]:
json.dump(args, open("merge_qwen2vl.json", "w", encoding="utf-8"), indent=2)

In [15]:
!llamafactory-cli export merge_qwen2vl.json

[INFO|configuration_utils.py:675] 2024-10-14 16:06:19,190 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2-VL-2B-Instruct/snapshots/aca78372505e6cb469c4fa6a35c60265b00ff5a4/config.json
[INFO|configuration_utils.py:742] 2024-10-14 16:06:19,193 >> Model config Qwen2VLConfig {
  "_name_or_path": "Qwen/Qwen2-VL-2B-Instruct",
  "architectures": [
    "Qwen2VLForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "image_token_id": 151655,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2_vl",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": {
    "mrope_section": [
      16,
      24,
      24
    ],
    "rope_type": "default",
    "type": "default"
  },
  "rope_th